In [ ]:
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/celeba

In [ ]:
!unzip /content/drive/MyDrive/img_align_celeba.zip -d /content/celeba

In [ ]:
import pathlib
data_dir = pathlib.Path('/content/celeba/img_align_celeba')
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*.jpg'), shuffle=False)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [128, 128])

def process_path(file_path):
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  img = tf.cast(img, 'float32')/255.
  return img, img

proc_ds = list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = proc_ds.take(20000).cache().batch(64, drop_remainder=True)
valid_ds = proc_ds.take(10000).cache().batch(64, drop_remainder=True)